In [ ]:
# 🧠 Adaptive Learning Coach — Interactive Demo  
### Agents + Tools + Memory | Assessment → Lesson → Quiz → Feedback → Evaluation

#This notebook showcases the full end-to-end AI tutoring loop using multi-agents and persistent memory.


In [1]:
%pip install sympy rich opentelemetry-sdk ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys, os
sys.path.append("../src")

from agents.assessment_agent import AssessmentAgent
from agents.lesson_agent import LessonAgent
from agents.quiz_agent import QuizAgent
from agents.feedback_agent import FeedbackAgent

from tools.persistence import init_db, load_memory
from eval.evaluator import run_evaluation

print("Imports loaded successfully.")


Imports loaded successfully.


In [3]:
db = init_db()

assessment_agent = AssessmentAgent(conn=db)
lesson_agent = LessonAgent(conn=db)
quiz_agent = QuizAgent(conn=db)
feedback_agent = FeedbackAgent(conn=db)

print("Agents initialized — DB ready.")


[green][OK] SQLite memory database initialized.[/]
Agents initialized — DB ready.


In [6]:
import ipywidgets as widgets
from IPython.display import display

user_box = widgets.Text(
    value="student_demo",
    description="User ID:",
    style={'description_width': 'initial'},
)

start_button = widgets.Button(
    description="Start Session",
    button_style='success'
)

display(user_box, start_button)

session_started = False
def start_click(_):
    global session_started
    session_started = True
    print(f"Session started for user: {user_box.value}")

start_button.on_click(start_click)


Text(value='student_demo', description='User ID:', style=TextStyle(description_width='initial'))

Button(button_style='success', description='Start Session', style=ButtonStyle())

In [7]:
if not session_started:
    print("Please run the previous cell and click Start Session.")
else:
    uid = user_box.value
    print(f"Running diagnostic assessment for: {uid}")

    questions = [
        "2*x + 3 = 11",
        "5*x - 4 = 21",
        "3*x + 9 = 0"
    ]

    print("\nAnswer the following:")
    answers = []
    for q in questions:
        print("•", q)
        answers.append(input("Your answer for x = "))

    assessment = assessment_agent.run_diagnostic(uid, answers)
    print("\n📌 Assessment Result:")
    assessment


Running diagnostic assessment for: student_demo

Answer the following:
• 2*x + 3 = 11
• 5*x - 4 = 21
• 3*x + 9 = 0


{"ts": "2025-11-27T16:31:01.843330Z", "level": "INFO", "msg": "intent_before_assessment", "logger": "assessment_agent", "trace_id": "assess-student_demo", "user_id": "student_demo"}
{"ts": "2025-11-27T16:31:01.936179Z", "level": "INFO", "msg": "question_graded", "logger": "assessment_agent", "trace_id": "assess-student_demo", "q_index": 0, "correct": false}
{"ts": "2025-11-27T16:31:01.941466Z", "level": "INFO", "msg": "question_graded", "logger": "assessment_agent", "trace_id": "assess-student_demo", "q_index": 1, "correct": false}
{"ts": "2025-11-27T16:31:01.946420Z", "level": "INFO", "msg": "question_graded", "logger": "assessment_agent", "trace_id": "assess-student_demo", "q_index": 2, "correct": false}


[cyan][OK] Memory loaded for student_demo[/]
[cyan][OK] Memory saved for student_demo[/]


{"ts": "2025-11-27T16:31:01.960764Z", "level": "INFO", "msg": "assessment_completed", "logger": "assessment_agent", "trace_id": "assess-student_demo", "user_id": "student_demo", "score": 0}



📌 Assessment Result:


In [8]:
lesson = lesson_agent.plan(uid, [assessment])
print("📘 Generated Lesson:")
lesson


{"ts": "2025-11-27T16:31:07.853101Z", "level": "INFO", "msg": "intent_before_lesson_plan", "logger": "lesson_agent", "trace_id": "lesson-student_demo", "user_id": "student_demo"}


[cyan][OK] Memory loaded for student_demo[/]
[cyan][OK] Memory saved for student_demo[/]


{"ts": "2025-11-27T16:31:07.867419Z", "level": "INFO", "msg": "lesson_planned", "logger": "lesson_agent", "trace_id": "lesson-student_demo", "user_id": "student_demo", "topic": "linear_equations", "difficulty": "Foundational"}


📘 Generated Lesson:


{'topic': 'linear_equations',
 'created_at': '2025-11-27T16:31:07.854506Z',
 'difficulty': 'Foundational',
 'learning_objectives': ['Isolate the variable x in single-variable linear equations',
  'Perform arithmetic operations on both sides of the equation',
  'Check solutions by substitution'],
 'focus': 'Begin with isolating the variable, move step-by-step, and verify each operation.',
 'short_explanation': 'To solve equations like a*x + b = c, first move constants to the right side by subtracting b, then divide by a to get x. Keep each step explicit.',
 'worked_example': {'equation_str': '4*x + (-4) = 8',
  'solution': 3.0,
  'steps': ['Start: 4*x + (-4) = 8',
   'Subtract -4 from both sides: 4*x = 12',
   'Divide both sides by 4: x = 3.0']},
 'practice_prompt': 'Solve 3 similar equations and check your steps. Try both positive and negative constants.',
 'score_prior': 0}

In [9]:
quiz = quiz_agent.generate_quiz(uid, lesson)
print("📝 Quiz Questions:")
user_answers = []

for item in quiz["questions"]:
    print("•", item["q"])
    user_answers.append(input("Your answer = "))

graded = quiz_agent.grade_quiz(uid, quiz, user_answers)
print("\n🏁 Quiz Results:")
graded


{"ts": "2025-11-27T16:31:12.126964Z", "level": "INFO", "msg": "intent_before_quiz_generate", "logger": "quiz_agent", "trace_id": "quiz-gen-student_demo", "user_id": "student_demo"}


[cyan][OK] Memory loaded for student_demo[/]
[cyan][OK] Memory saved for student_demo[/]


{"ts": "2025-11-27T16:31:12.137152Z", "level": "INFO", "msg": "quiz_generated", "logger": "quiz_agent", "trace_id": "quiz-gen-student_demo", "user_id": "student_demo", "num_q": 3}


📝 Quiz Questions:
• Solve for x: 4*x + (-4) = 8
• Solve for x: 4*x + (-4) = 10
• Solve for x: 5*x + (-4)  = 8


{"ts": "2025-11-27T16:31:18.329949Z", "level": "INFO", "msg": "intent_before_quiz_grade", "logger": "quiz_agent", "trace_id": "quiz-grade-student_demo", "user_id": "student_demo"}
{"ts": "2025-11-27T16:31:18.333874Z", "level": "INFO", "msg": "quiz_question_graded", "logger": "quiz_agent", "trace_id": "quiz-grade-student_demo", "q_index": 0, "correct": false}
{"ts": "2025-11-27T16:31:18.338639Z", "level": "INFO", "msg": "quiz_question_graded", "logger": "quiz_agent", "trace_id": "quiz-grade-student_demo", "q_index": 1, "correct": false}
{"ts": "2025-11-27T16:31:18.343501Z", "level": "INFO", "msg": "quiz_question_graded", "logger": "quiz_agent", "trace_id": "quiz-grade-student_demo", "q_index": 2, "correct": false}


[cyan][OK] Memory loaded for student_demo[/]
[cyan][OK] Memory saved for student_demo[/]


{"ts": "2025-11-27T16:31:18.354660Z", "level": "INFO", "msg": "quiz_graded", "logger": "quiz_agent", "trace_id": "quiz-grade-student_demo", "user_id": "student_demo", "score": 0, "new_mastery": 0}



🏁 Quiz Results:


{'user_id': 'student_demo',
 'timestamp': '2025-11-27T16:31:18.344915Z',
 'per_question': [{'q_index': 0,
   'question': 'Solve for x: 4*x + (-4) = 8',
   'expected': 3.0,
   'user_answer_raw': '4',
   'user_answer_parsed': 4.0,
   'correct': False,
   'explanation': 'Incorrect — expected 3.0, got 4.0.'},
  {'q_index': 1,
   'question': 'Solve for x: 4*x + (-4) = 10',
   'expected': 3.5,
   'user_answer_raw': '3',
   'user_answer_parsed': 3.0,
   'correct': False,
   'explanation': 'Incorrect — expected 3.5, got 3.0.'},
  {'q_index': 2,
   'question': 'Solve for x: 5*x + (-4)  = 8',
   'expected': 2.4,
   'user_answer_raw': '2',
   'user_answer_parsed': 2.0,
   'correct': False,
   'explanation': 'Incorrect — expected 2.4, got 2.0.'}],
 'correct_count': 0,
 'total_questions': 3,
 'score_percent': 0}

In [10]:
fb = feedback_agent.provide_feedback(uid, graded)
print("💬 Feedback & Recommendations:")
fb


{"ts": "2025-11-27T16:31:35.641868Z", "level": "INFO", "msg": "intent_before_feedback", "logger": "feedback_agent", "trace_id": "feedback-student_demo", "user_id": "student_demo"}


{
    "name": "feedback-generation",
    "context": {
        "trace_id": "0x042da78f7913c9029cb5d254370893c2",
        "span_id": "0x4af87f50f36728f6",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-11-27T16:31:35.643611Z",
    "end_time": "2025-11-27T16:31:35.649026Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.38.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
[cyan][OK] Memory loaded for student_demo[/]
[cyan][OK] Memory saved for student_demo[/]


{"ts": "2025-11-27T16:31:35.660377Z", "level": "INFO", "msg": "feedback_saved", "logger": "feedback_agent", "trace_id": "feedback-student_demo", "user_id": "student_demo", "score": 0}


💬 Feedback & Recommendations:


{'user_id': 'student_demo',
 'timestamp': '2025-11-27T16:31:35.650959Z',
 'quiz_score': 0,
 'items': [{'q_index': 0,
   'status': 'incorrect',
   'message': 'See step-by-step guidance and hint below.',
   'details': {'question': 'Solve for x: 4*x + (-4) = 8',
    'expected_expr': 3.0,
    'expected_value': None,
    'user_value': 4.0,
    'steps': ['Target question: Solve for x: 4*x + (-4) = 8',
     'Target equation: 3.0',
     'Could not compute expected solution deterministically.',
     'Checklist:',
     '1) Move the constant term to the right side (subtract/add).',
     '2) Divide both sides by the coefficient of x.',
     '3) Substitute your solution back to check.'],
    'hint': 'Verify you first subtracted/added the constant term, then divided by the coefficient. Show each step.'}},
  {'q_index': 1,
   'status': 'incorrect',
   'message': 'See step-by-step guidance and hint below.',
   'details': {'question': 'Solve for x: 4*x + (-4) = 10',
    'expected_expr': 3.5,
    'expec

In [11]:
print("🗂 Memory snapshot:")
load_memory(db, uid)


🗂 Memory snapshot:
[cyan][OK] Memory loaded for student_demo[/]


{'diagnostics': [{'user_id': 'student_demo',
   'timestamp': None,
   'per_question': [{'q_index': 0,
     'question': 'Solve for x: 2*x + 3 = 11',
     'expected': 4.0,
     'user_answer_raw': '5',
     'user_answer_parsed': 5.0,
     'correct': False,
     'explanation': 'Incorrect — expected 4.0, got 5.0.'},
    {'q_index': 1,
     'question': 'Solve for x: 5*x - 4 = 21',
     'expected': 5.0,
     'user_answer_raw': '4',
     'user_answer_parsed': 4.0,
     'correct': False,
     'explanation': 'Incorrect — expected 5.0, got 4.0.'},
    {'q_index': 2,
     'question': 'Solve for x: 3*x + 9 = 0',
     'expected': -3.0,
     'user_answer_raw': '3',
     'user_answer_parsed': 3.0,
     'correct': False,
     'explanation': 'Incorrect — expected -3.0, got 3.0.'}],
   'correct_count': 0,
   'total_questions': 3,
   'score_percent': 0},
  {'user_id': 'student_demo',
   'timestamp': None,
   'per_question': [{'q_index': 0,
     'question': 'Solve for x: 2*x + 3 = 11',
     'expected': 4.0

In [ ]:
import sys, os

ROOT = os.path.abspath("../") 
if ROOT not in sys.path:
    sys.path.append(ROOT)

print("Project root added to sys.path:", ROOT)


Project root added to sys.path: d:\Capstone Project\adaptive-coach


In [13]:
print("📊 Evaluation report:")
from src.eval.evaluator import run_evaluation
evaluator = run_evaluation(conn=None, golden_path="../src/eval/golden_cases.json")
evaluator



📊 Evaluation report:
[green][OK] SQLite memory database initialized.[/]
[yellow][OK] No memory found for student_test_01. Returning empty object.[/]


{"ts": "2025-11-27T16:31:49.754914Z", "level": "INFO", "msg": "eval_case_result", "logger": "evaluator", "case_id": "case-001", "user_id": "student_test_01", "actual": {"score_percent": 0, "pre_score": 33}, "judgement": {"score": 0, "comment": "Failed: post 0 pre 33 delta -33 (needs >= 10)", "passed": false}}


[yellow][OK] No memory found for student_test_02. Returning empty object.[/]


{"ts": "2025-11-27T16:31:49.757589Z", "level": "INFO", "msg": "eval_case_result", "logger": "evaluator", "case_id": "case-002", "user_id": "student_test_02", "actual": {"score_percent": 0, "pre_score": 66}, "judgement": {"score": 0, "comment": "Failed: post 0 pre 66 delta -66 (needs >= 0)", "passed": false}}


[{'case_id': 'case-001',
  'user_id': 'student_test_01',
  'actual': {'score_percent': 0, 'pre_score': 33},
  'judgement': {'score': 0,
   'comment': 'Failed: post 0 pre 33 delta -33 (needs >= 10)',
   'passed': False}},
 {'case_id': 'case-002',
  'user_id': 'student_test_02',
  'actual': {'score_percent': 0, 'pre_score': 66},
  'judgement': {'score': 0,
   'comment': 'Failed: post 0 pre 66 delta -66 (needs >= 0)',
   'passed': False}}]